# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_file = ("weather_data.csv")

cities_df = pd.read_csv(csv_file)
city_df = cities_df.dropna()
city_df

,Unnamed: 0,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tutoia,-2.7619,-42.2744,91.38,66.0,15.0,15.59,BR,1.638897e+09
1,1,severo-kurilsk,50.6789,156.1250,30.16,62.0,95.0,27.56,RU,1.638897e+09
2,2,bongandanga,1.5000,21.0500,72.36,96.0,99.0,2.08,CD,1.638897e+09
3,3,lasa,46.6166,10.7002,28.31,64.0,5.0,2.91,IT,1.638897e+09
4,4,uniontown,39.9001,-79.7164,32.45,44.0,97.0,4.00,US,1.638897e+09
...,...,...,...,...,...,...,...,...,...,...
572,572,quepos,60.1247,64.7758,-1.91,92.0,79.0,6.53,RU,1.638898e+09
573,573,whitehorse,-14.3779,34.3332,69.69,37.0,63.0,3.11,MW,1.638898e+09
574,574,taitung,64.8621,11.2373,23.11,58.0,0.0,19.57,NO,1.638898e+09
575,575,parana,43.2128,-75.4557,35.46,87.0,90.0,8.05,US,1.638898e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, 
                                 max_intensity = 100, point_radius = 5)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_df = city_df[(city_df['Max_Temp'] >= 75) & (city_df['Max_Temp'] <= 90)]
ideal_df = ideal_df[ideal_df['Wind Speed'] <= 10]
ideal_df = ideal_df[ideal_df['Cloudiness'] <= 0]
ideal_df = ideal_df[ideal_df['Humidity'] <= 70]
hotel_df = ideal_df
hotel_df

,Unnamed: 0,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind Speed,Country,Date
167,167,cabo san lucas,17.0151,54.0924,75.29,50.0,0.0,4.61,OM,1.638898e+09
340,340,hjorring,-45.5752,-72.0662,80.55,29.0,0.0,6.91,CL,1.638898e+09
448,448,vryburg,-36.6167,-71.8333,83.39,17.0,0.0,8.25,CL,1.638898e+09
449,449,ucluelet,10.5669,72.6420,80.29,69.0,0.0,9.57,IN,1.638898e+09
482,482,wakkanai,-34.6534,-58.6198,85.01,45.0,0.0,7.00,AR,1.638898e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df['Hotel Name'] = ""
hotel_df

,Unnamed: 0,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
167,167,cabo san lucas,17.0151,54.0924,75.29,50.0,0.0,4.61,OM,1.638898e+09,
340,340,hjorring,-45.5752,-72.0662,80.55,29.0,0.0,6.91,CL,1.638898e+09,
448,448,vryburg,-36.6167,-71.8333,83.39,17.0,0.0,8.25,CL,1.638898e+09,
449,449,ucluelet,10.5669,72.6420,80.29,69.0,0.0,9.57,IN,1.638898e+09,
482,482,wakkanai,-34.6534,-58.6198,85.01,45.0,0.0,7.00,AR,1.638898e+09,


In [7]:
for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }
        lat = row['Lat']
        lng = row['Lng']

        params['location'] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df 

,Unnamed: 0,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
167,167,cabo san lucas,17.0151,54.0924,75.29,50.0,0.0,4.61,OM,1.638898e+09,Salalah Gardens Hotel
340,340,hjorring,-45.5752,-72.0662,80.55,29.0,0.0,6.91,CL,1.638898e+09,El Reloj Hotel by DOT Boutique
448,448,vryburg,-36.6167,-71.8333,83.39,17.0,0.0,8.25,CL,1.638898e+09,NaN
449,449,ucluelet,10.5669,72.6420,80.29,69.0,0.0,9.57,IN,1.638898e+09,Bismillah Hotel And Lodging
482,482,wakkanai,-34.6534,-58.6198,85.01,45.0,0.0,7.00,AR,1.638898e+09,Hotel Haedo


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig
# Display figure

Figure(layout=FigureLayout(height='420px'))